In [146]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import csv


In [147]:
# Top 100 Anime of all time according to stakeholder
url = "https://www.imdb.com/list/ls057577566/"
response = requests.get(url)
soup = BeautifulSoup(response.content,'html.parser')

In [148]:
# Create lists for data to be scraped into
name = []   # name of the anime
rating = []  # rating of the anime
aired_duration = [] # released year - end year
average_runtime = [] # average duration of each anime episode
genre = [] # genre of the anime
MPAA_rating = [] # MPAA rating
votes = []

In [149]:
# Use bs4 to place all 100 anime tv-shows into a list
anime_data = soup.find_all('div', attrs={'class':'lister-item mode-detail'})

In [150]:
# iterate through all 100 movies
for data in anime_data:
    # extract all the names of the animes
    name.append(data.h3.a.text)  

    # extract all the aired_duration of the animes
    aired_duration.append(data.find(string=re.compile(r'\(.*\)')))

    # extract all the genres of the animes
    genre.append(data.p.find('span', attrs={'class':'genre'}).text)

    # extract all the mpaa certificates of the animes
    mpaa_review = data.p.find('span',class_='certificate')
    if mpaa_review:
        MPAA_rating.append(mpaa_review.text) 
    else:
        MPAA_rating.append('None')

    # extract all the average run time of the animes
    runtime_span = data.p.find('span', class_='runtime')
    if runtime_span:
        average_runtime.append(runtime_span.text)
    else:
        average_runtime.append("None")

    # extract all the ratings of the animes
    rate = data.find('span',class_='ipl-rating-star__rating')
    if rate:
        rating.append(rate.text)
    else:
        rating.append('None')

    # extract all the votes of the animes
    vote = data.find('span', attrs={'name':'nv'})
    if vote:
        votes.append(vote.text)
    else:
        votes.append("None")


In [151]:
# cleaned the genre array by stripping the whitespaces and newlines within each element
temp_genre = []
for x in genre:
    temp_genre.append(x.strip())
genre = temp_genre

In [152]:
# cleaned the aired_duration array by removing all the parentheses from each element
aired_duration = [s.replace('(', '').replace(')', '') for s in  aired_duration]

In [153]:
# cleaned the average_runtime array by removing the minutes part from each element
average_runtime  = [int(t.replace(' min', '')) if t != "None" else "None" for t in average_runtime]

In [154]:
# write all the columns into the csv file
with open("scraped_top_100_anime.csv", "w", newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["title", "rating", "aired_duration","average_runtime", "genre","MPAA_rating", "votes"])
    for title, score, duration, avg_time, genres, mpaa, vote in zip(name,rating,aired_duration,average_runtime,genre,MPAA_rating,votes):
       # print(title + " " + score+ " " + duration + " " + avg_time + " " + genres + " "  + mpaa + " " + vote)
        writer.writerow([title, score, duration, avg_time, genres, mpaa, vote])
